In [1]:
import time
import string
from collections import namedtuple

import numpy as np
import tensorflow as tf
from keras.preprocessing.text import text_to_word_sequence
from keras.utils import to_categorical
from keras.models import Sequential
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense,LSTM,Dropout,Embedding

Using TensorFlow backend.


In [0]:
# Read the data
with open('The_Da_Vinci_Code.txt', encoding="utf8", errors='ignore') as f:
    text=f.read()

In [0]:
def clean_doc(doc):
	# replace '--' with a space ' '
	doc = doc.replace('--', ' ')
	# split into tokens by white space
	tokens = doc.split()
	# remove punctuation from each token
	table = str.maketrans('', '', string.punctuation)
	tokens = [w.translate(table) for w in tokens]
	# remove remaining tokens that are not alphabetic
	tokens = [word for word in tokens if word.isalpha()]
	# make lower case
	tokens = [word.lower() for word in tokens]
	return tokens

In [0]:
# Get text as tokens
tokens = clean_doc(text)
# vocabulary size
total_words = len(tokens)

In [0]:
# Length of unique tokens
unique_words =len(set(tokens))

In [0]:
# Integer encode the tokens
vocab = sorted(set(tokens))
vocab_to_int = {c: i for i, c in enumerate(vocab)}
int_to_vocab = dict(enumerate(vocab))
encoded = np.array([vocab_to_int[c] for c in tokens], dtype=np.int32)

In [0]:
np.save('int_to_vocab.npy', int_to_vocab)
np.save('vocab_to_int.npy', vocab_to_int)

In [8]:
tokens[:20]

['the',
 'da',
 'vinci',
 'code',
 'dan',
 'brown',
 'for',
 'blythe',
 'again',
 'more',
 'than',
 'ever',
 'prologue',
 'louvre',
 'museum',
 'paris',
 'pm',
 'renowned',
 'curator',
 'jacques']

In [0]:
from keras.utils import to_categorical
seq_len = 10
X=[]
y = np.zeros((total_words-seq_len,unique_words), dtype=np.bool)
for i in range(seq_len,total_words):
  words_in = tokens[i-seq_len:i]
  word_out = tokens[i]
  X.append([vocab_to_int[word] for word in words_in])
  y[i-seq_len,vocab_to_int[word_out]]=1

In [0]:
X = np.asarray(X)

In [11]:
X.shape

(137995, 10)

In [12]:
# define model
model = Sequential()
model.add(Embedding(X.shape[0], 10, input_length=seq_len))
model.add(LSTM(512))
model.add(Dropout(0.5))
model.add(Dense(y.shape[1], activation='softmax'))
print(model.summary())
# compile model
model.compile(loss='categorical_crossentropy', optimizer="Adam")
# fit model
filepath='best_model.h5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
model.fit(X,y, batch_size=128, epochs=100,callbacks=[checkpoint])
# save the model to file

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 10, 10)            1379950   
_________________________________________________________________
lstm_1 (LSTM)                (None, 512)               1071104   
_________________________________________________________________
dropout_1 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 11495)             5896935   
Total params: 8,347,989
Trainable params: 8,347,989
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/100
 13312/137995 [=>............................] - ETA: 1:06 - loss: 7.4697

137995/137995 [==============================] - 60s 433us/step - loss: 6.9497

Epoch 00001: loss improved from inf to 6.94970, saving model to best_model.h5
Epoch 2/100
 15616/137995 [==>...........................] - ETA: 51s - loss: 6.6987

137995/137995 [==============================] - 58s 423us/step - loss: 6.6756

Epoch 00002: loss improved from 6.94970 to 6.67559, saving model to best_model.h5
Epoch 3/100
 16128/137995 [==>...........................] - ETA: 51s - loss: 6.5147

137995/137995 [==============================] - 58s 423us/step - loss: 6.4997

Epoch 00003: loss improved from 6.67559 to 6.49972, saving model to best_model.h5
Epoch 4/100
 16128/137995 [==>...........................] - ETA: 51s - loss: 6.3252

137995/137995 [==============================] - 58s 422us/step - loss: 6.3159

Epoch 00004: loss improved from 6.49972 to 6.31588, saving model to best_model.h5
Epoch 5/100
 16512/137995 [==>...........................] - ETA: 51s - loss: 6.1066

...

137995/137995 [==============================] - 59s 425us/step - loss: 1.1061

Epoch 00096: loss improved from 1.11072 to 1.10606, saving model to best_model.h5
Epoch 97/100
 17536/137995 [==>...........................] - ETA: 51s - loss: 1.0340

137995/137995 [==============================] - 59s 424us/step - loss: 1.1073

Epoch 00097: loss did not improve
Epoch 98/100
 20608/137995 [===>..........................] - ETA: 50s - loss: 0.9898

137995/137995 [==============================] - 59s 424us/step - loss: 1.1000

Epoch 00098: loss improved from 1.10606 to 1.10004, saving model to best_model.h5
Epoch 99/100
 17536/137995 [==>...........................] - ETA: 50s - loss: 0.9795

137995/137995 [==============================] - 58s 424us/step - loss: 1.1032

Epoch 00099: loss did not improve
Epoch 100/100
 20608/137995 [===>..........................] - ETA: 49s - loss: 1.0120

137995/137995 [==============================] - 59s 427us/step - loss: 1.0948

Epoch 00100: loss improved from 1.10004 to 1.09481, saving model to best_model.h5


In [0]:
from google.colab import files
files.download('best_model.h5')

In [0]:
from keras.models import load_model
model=load_model('best_model.h5')

In [0]:
from keras.preprocessing.sequence import pad_sequences
def generate_seq(model, seq_length, seed_text, n_words):
    result = list()
    in_text = seed_text
    # generate a fixed number of words
    for _ in range(n_words):
      encoded=[]
      # encode the text as integer
      for word in in_text.split():
        encoded.append(vocab_to_int[word])
      # truncate sequences to a fixed length
      encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
      # predict probabilities for each word
      yhat = model.predict_classes(encoded, verbose=0)
      # map predicted word index to word
      out_word = ''
      for word, index in vocab_to_int.items():
        if index == yhat:
          out_word = word
          break
      # append to input
      in_text += ' ' + out_word
      result.append(out_word)
    return ' '.join(result)

In [16]:
seed=' '.join(tokens[:100])
generated = generate_seq(model,seq_len,seed,1000)
print(generated)

a moment gasping for breath taking stock i am still alive he crawled out from under the canvas and scanned the cavernous space for someplace to hide a voice spoke chillingly close do not move on the floor and then it is the knights thing of the priory of sion and the grail that is the holy grail that the documents simply had been told the grail in the chapel she was certain he had just yet the line of the painful fallingout she was not in the moment he thought she had been given the teacher to the end of days a little sophie and be finished in the side of the church and a lone thought of wine and the most feminine of this of the most world the holy grail is a grail but when the documents documents a knight that the grail stopped at the tomb of the french police langdon looked back at the mona lisa before the door im a message with the keystone he felt a startled and that news teabing had been often in the hurry of the priory and yet the true man was not being one is the holy grail is a